In [1]:
import sys
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import os
import gc
import math
import numpy as np
from collections import Counter
from util import ShowProcess
import scipy.sparse as sp

## Import user features, ad features, train dataset, test dataset

In [4]:
%%time
if os.path.exists('./preliminary_contest_data/userFeature.csv'):
    print('load')
    user_feature=pd.read_csv('./preliminary_contest_data/userFeature.csv')
else:
    print('process')
    userFeature_data = []
    with open('./preliminary_contest_data/userFeature.data', 'r') as f:
        for i, line in enumerate(f):
            line = line.strip().split('|')
            userFeature_dict = {}
            for each in line:
                each_list = each.split(' ')
                userFeature_dict[each_list[0]] = ' '.join(each_list[1:])
            userFeature_data.append(userFeature_dict)
            sp.show_process()
        sp.close()
        user_feature = pd.DataFrame(userFeature_data)
        user_feature.to_csv('./preliminary_contest_data/userFeature.csv', index=False)

load
CPU times: user 1min 3s, sys: 10.1 s, total: 1min 13s
Wall time: 1min 4s


In [63]:
ad_feature = pd.read_csv('./preliminary_contest_data/adFeature.csv')
train=pd.read_csv('./preliminary_contest_data/train.csv')
predict=pd.read_csv('./preliminary_contest_data/test1.csv')

## Merge data

In [7]:
%%time
train.loc[train['label']==-1,'label']=0
predict['label']=-1

data=pd.concat([train,predict])
data=pd.merge(data,ad_feature,on='aid',how='left')
data=pd.merge(data,user_feature,on='uid',how='left')

In [8]:
data.head(2)

,aid,uid,label,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType,...,interest4,interest5,kw1,kw2,kw3,marriageStatus,os,topic1,topic2,topic3
0,699,78508957,0,1082,295940,731679,59,13,0,6,...,NaN,NaN,124532 762727 771871 8561 610048,NaN,NaN,10,0,4908 9745 7070 4203 3292,NaN,NaN
1,1991,3637295,0,702,42104,1441131,53,10,4669,11,...,NaN,100 131 37 8 46,531499 542834 303322 503318 68211,89232 30101 117462 117548 85039,NaN,13,1,8445 6762 7608 523 7688,4518 862 3643 702 5247,NaN


## If merge data exist, read, else to save a copy of merge data

In [2]:
%%time
if os.path.exists('./merge_data.csv'):
    print('load')
    data = pd.read_csv('./merge_data.csv')
    data = data.fillna('-1')
else:
    print('save')
    data = data.fillna('-1')
    data.to_csv('./merge_data.csv')

load
CPU times: user 2min 2s, sys: 24.9 s, total: 2min 27s
Wall time: 1min 43s


In [20]:
# %%time
# data = data.fillna('-1')

In [21]:
# data.isnull().sum(axis=0)

## Processing features one hot/vectorize features

In [4]:
one_hot_feature=['LBS','age','carrier','consumptionAbility','education','gender','house','os','ct',
                 'marriageStatus','advertiserId','campaignId', 'creativeId',
                 'adCategoryId', 'productId', 'productType']
vector_feature=['appIdAction','appIdInstall','interest1','interest2','interest3','interest4','interest5',
                'kw1','kw2','kw3','topic1','topic2','topic3']

### Processing one-hot features

In [5]:
%%time
sp = ShowProcess(len(one_hot_feature))
for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])
    sp.show_process()
sp.close()

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
done
CPU times: user 5min, sys: 45.2 s, total: 5min 46s
Wall time: 2min 30s


In [6]:
%%time
enc = OneHotEncoder()
categorical_features = data[one_hot_feature].values
one_hot_features = enc.fit_transform(categorical_features)

CPU times: user 28.7 s, sys: 31.2 s, total: 60 s
Wall time: 17.2 s


In [7]:
print('type of set is ',type(one_hot_features), 'shape of array is ',one_hot_features.shape)

type of set is  <class 'scipy.sparse.csr.csr_matrix'> shape of array is  (11064803, 1445)


### Processing vectorize features

In [8]:
%%time
vectorize_features = []
sp = ShowProcess(len(vector_feature))
for feature in vector_feature:
    feature_df = CountVectorizer().fit_transform(data[feature])
    vectorize_features.append(feature_df)
    sp.show_process()
sp.close()

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
done
CPU times: user 14min 38s, sys: 42.6 s, total: 15min 21s
Wall time: 15min 21s


In [10]:
%%time
import scipy.sparse as sp
vectorize_features_merge = sp.hstack((vectorize_features[0],vectorize_features[1]))
for i in range(2, len(vectorize_features), 1):
    vectorize_features_merge = sp.hstack((vectorize_features_merge, vectorize_features[i]))

CPU times: user 6min 22s, sys: 3min 20s, total: 9min 42s
Wall time: 1min 11s


In [11]:
%%time
vectorize_features_merge = vectorize_features_merge.tocsr()

CPU times: user 34 s, sys: 6.61 s, total: 40.6 s
Wall time: 40.6 s


In [26]:
print('type of merged features is ',type(vectorize_features_merge),'shape is ', vectorize_features_merge.shape)

type of merged features is  <class 'scipy.sparse.csr.csr_matrix'> shape is  (11064803, 418206)


## Train, test split

In [27]:
%%time
label = data['label']
train_y = label[np.where(label != -1)[0]].values
split_index = len(train_y)

CPU times: user 2.58 s, sys: 417 ms, total: 3 s
Wall time: 2.99 s


In [31]:
len(train_y)

8798814

In [15]:
type(train_y), train_y.shape

(numpy.ndarray, (8798814,))

In [32]:
%%time
train_one_hot_matrix = one_hot_features[:split_index]
train_vectorize_matrix = vectorize_features_merge[:split_index]

test_one_hot_matrix = one_hot_features[split_index:]
test_vectorize_matrix = vectorize_features_merge[split_index:]

CPU times: user 6.41 s, sys: 10.7 s, total: 17.1 s
Wall time: 17.1 s


In [33]:
%%time
train_x = data[['creativeSize']][:split_index]
train_x = sp.csr_matrix(train_x)

train_x = sp.hstack((train_x, train_one_hot_matrix))
train_x = sp.hstack((train_x, train_vectorize_matrix))
train_x = train_x.tocsr()

KeyboardInterrupt: 

In [34]:
type(train_x), train_x.shape

(scipy.sparse.coo.coo_matrix, (8798814, 419652))

In [35]:
%%time
test_x = data[['creativeSize']][split_index:]
test_x = sp.csr_matrix(test_x)

test_x = sp.hstack((test_x, test_one_hot_matrix))
test_x = sp.hstack((test_x, test_vectorize_matrix))
test_x = test_x.tocsr()

CPU times: user 51.2 s, sys: 17.3 s, total: 1min 8s
Wall time: 13.7 s


In [36]:
type(test_x), test_x.shape

(scipy.sparse.csr.csr_matrix, (2265989, 419652))

## Save vectorize features

In [21]:
%%time
sp.save_npz('./preliminary_contest_data/train_x.npz', train_x)
sp.save_npz('./preliminary_contest_data/test_x.npz', test_x)

np.save('./preliminary_contest_data/train_y.npy', train_y)

CPU times: user 4min 1s, sys: 5.64 s, total: 4min 7s
Wall time: 4min 7s


In [22]:
train_x.shape

(8798814, 419652)

## Read train,test set

In [84]:
train=sparse.load_npz('./preliminary_contest_data/train_x.npz')
train_y=np.load('./preliminary_contest_data/train_y.npy')
test_x=sparse.load_npz('./preliminary_contest_data/test_x.npz')

In [85]:
from sklearn.model_selection import train_test_split
train_x, evals_x, train_y, evals_y = train_test_split(train,train_y,test_size=0.2, random_state=2018)

In [61]:
import lightgbm as lgb

print("LGB test")
clf = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
    max_depth=-1, n_estimators=10000, objective='binary',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, min_child_weight=50, random_state=2018, n_jobs=-1
)
clf.fit(train_x, train_y, eval_set=[(evals_x, evals_y)], eval_metric='auc',early_stopping_rounds=2000)

LGB test
[1]	valid_0's auc: 0.628548
Training until validation scores don't improve for 2000 rounds.
[2]	valid_0's auc: 0.632528
[3]	valid_0's auc: 0.64005
[4]	valid_0's auc: 0.645677
[5]	valid_0's auc: 0.647953
[6]	valid_0's auc: 0.651385
[7]	valid_0's auc: 0.650888
[8]	valid_0's auc: 0.651366
[9]	valid_0's auc: 0.653486
[10]	valid_0's auc: 0.65533
[11]	valid_0's auc: 0.658197
[12]	valid_0's auc: 0.659296
[13]	valid_0's auc: 0.659577
[14]	valid_0's auc: 0.659474
[15]	valid_0's auc: 0.659595
[16]	valid_0's auc: 0.660356
[17]	valid_0's auc: 0.661035
[18]	valid_0's auc: 0.660529
[19]	valid_0's auc: 0.660036
[20]	valid_0's auc: 0.66086
[21]	valid_0's auc: 0.660819
[22]	valid_0's auc: 0.661736
[23]	valid_0's auc: 0.662522
[24]	valid_0's auc: 0.663305
[25]	valid_0's auc: 0.663441
[26]	valid_0's auc: 0.663522
[27]	valid_0's auc: 0.664158
[28]	valid_0's auc: 0.664726
[29]	valid_0's auc: 0.665156
[30]	valid_0's auc: 0.665685
[31]	valid_0's auc: 0.665696
[32]	valid_0's auc: 0.666187
[33]	valid_

[277]	valid_0's auc: 0.718842
[278]	valid_0's auc: 0.718891
[279]	valid_0's auc: 0.718953
[280]	valid_0's auc: 0.718992
[281]	valid_0's auc: 0.719069
[282]	valid_0's auc: 0.719136
[283]	valid_0's auc: 0.719185
[284]	valid_0's auc: 0.719259
[285]	valid_0's auc: 0.719312
[286]	valid_0's auc: 0.719357
[287]	valid_0's auc: 0.719413
[288]	valid_0's auc: 0.719443
[289]	valid_0's auc: 0.719504
[290]	valid_0's auc: 0.71955
[291]	valid_0's auc: 0.7196
[292]	valid_0's auc: 0.719632
[293]	valid_0's auc: 0.719769
[294]	valid_0's auc: 0.719817
[295]	valid_0's auc: 0.719877
[296]	valid_0's auc: 0.719931
[297]	valid_0's auc: 0.719973
[298]	valid_0's auc: 0.720021
[299]	valid_0's auc: 0.72008
[300]	valid_0's auc: 0.720127
[301]	valid_0's auc: 0.720182
[302]	valid_0's auc: 0.720211
[303]	valid_0's auc: 0.72024
[304]	valid_0's auc: 0.720403
[305]	valid_0's auc: 0.720442
[306]	valid_0's auc: 0.72049
[307]	valid_0's auc: 0.720548
[308]	valid_0's auc: 0.720613
[309]	valid_0's auc: 0.720672
[310]	valid_0's 

[552]	valid_0's auc: 0.730285
[553]	valid_0's auc: 0.730302
[554]	valid_0's auc: 0.730325
[555]	valid_0's auc: 0.730347
[556]	valid_0's auc: 0.730366
[557]	valid_0's auc: 0.730385
[558]	valid_0's auc: 0.730413
[559]	valid_0's auc: 0.730442
[560]	valid_0's auc: 0.730509
[561]	valid_0's auc: 0.730547
[562]	valid_0's auc: 0.730577
[563]	valid_0's auc: 0.730595
[564]	valid_0's auc: 0.73061
[565]	valid_0's auc: 0.730634
[566]	valid_0's auc: 0.730648
[567]	valid_0's auc: 0.730657
[568]	valid_0's auc: 0.730681
[569]	valid_0's auc: 0.73075
[570]	valid_0's auc: 0.730785
[571]	valid_0's auc: 0.730805
[572]	valid_0's auc: 0.730831
[573]	valid_0's auc: 0.730849
[574]	valid_0's auc: 0.73087
[575]	valid_0's auc: 0.73089
[576]	valid_0's auc: 0.730906
[577]	valid_0's auc: 0.730916
[578]	valid_0's auc: 0.730929
[579]	valid_0's auc: 0.730935
[580]	valid_0's auc: 0.730957
[581]	valid_0's auc: 0.730978
[582]	valid_0's auc: 0.731001
[583]	valid_0's auc: 0.731021
[584]	valid_0's auc: 0.731028
[585]	valid_0'

[827]	valid_0's auc: 0.735155
[828]	valid_0's auc: 0.735165
[829]	valid_0's auc: 0.735177
[830]	valid_0's auc: 0.735191
[831]	valid_0's auc: 0.735221
[832]	valid_0's auc: 0.735226
[833]	valid_0's auc: 0.735231
[834]	valid_0's auc: 0.735259
[835]	valid_0's auc: 0.735266
[836]	valid_0's auc: 0.735274
[837]	valid_0's auc: 0.735279
[838]	valid_0's auc: 0.735279
[839]	valid_0's auc: 0.735287
[840]	valid_0's auc: 0.73529
[841]	valid_0's auc: 0.735293
[842]	valid_0's auc: 0.735301
[843]	valid_0's auc: 0.735318
[844]	valid_0's auc: 0.735323
[845]	valid_0's auc: 0.735326
[846]	valid_0's auc: 0.73533
[847]	valid_0's auc: 0.735366
[848]	valid_0's auc: 0.735377
[849]	valid_0's auc: 0.735392
[850]	valid_0's auc: 0.735416
[851]	valid_0's auc: 0.735428
[852]	valid_0's auc: 0.735442
[853]	valid_0's auc: 0.735464
[854]	valid_0's auc: 0.735485
[855]	valid_0's auc: 0.735499
[856]	valid_0's auc: 0.735505
[857]	valid_0's auc: 0.735515
[858]	valid_0's auc: 0.735518
[859]	valid_0's auc: 0.735528
[860]	valid_

[1098]	valid_0's auc: 0.73786
[1099]	valid_0's auc: 0.737863
[1100]	valid_0's auc: 0.73787
[1101]	valid_0's auc: 0.737869
[1102]	valid_0's auc: 0.737876
[1103]	valid_0's auc: 0.737892
[1104]	valid_0's auc: 0.737893
[1105]	valid_0's auc: 0.737911
[1106]	valid_0's auc: 0.737913
[1107]	valid_0's auc: 0.737914
[1108]	valid_0's auc: 0.737924
[1109]	valid_0's auc: 0.737958
[1110]	valid_0's auc: 0.737966
[1111]	valid_0's auc: 0.737972
[1112]	valid_0's auc: 0.737981
[1113]	valid_0's auc: 0.73798
[1114]	valid_0's auc: 0.737986
[1115]	valid_0's auc: 0.737996
[1116]	valid_0's auc: 0.737999
[1117]	valid_0's auc: 0.738018
[1118]	valid_0's auc: 0.738027
[1119]	valid_0's auc: 0.738037
[1120]	valid_0's auc: 0.738041
[1121]	valid_0's auc: 0.73805
[1122]	valid_0's auc: 0.738057
[1123]	valid_0's auc: 0.738101
[1124]	valid_0's auc: 0.738105
[1125]	valid_0's auc: 0.738112
[1126]	valid_0's auc: 0.73811
[1127]	valid_0's auc: 0.738126
[1128]	valid_0's auc: 0.738139
[1129]	valid_0's auc: 0.738143
[1130]	valid_

...

[9078]	valid_0's auc: 0.746411
[9079]	valid_0's auc: 0.74641
[9080]	valid_0's auc: 0.746411
[9081]	valid_0's auc: 0.746412
[9082]	valid_0's auc: 0.746412
[9083]	valid_0's auc: 0.74641
[9084]	valid_0's auc: 0.746409
[9085]	valid_0's auc: 0.746408
[9086]	valid_0's auc: 0.746408
[9087]	valid_0's auc: 0.746408
[9088]	valid_0's auc: 0.746407
[9089]	valid_0's auc: 0.746411
[9090]	valid_0's auc: 0.746411
[9091]	valid_0's auc: 0.74641
[9092]	valid_0's auc: 0.746409
[9093]	valid_0's auc: 0.746412
[9094]	valid_0's auc: 0.746413
[9095]	valid_0's auc: 0.746415
[9096]	valid_0's auc: 0.746413
[9097]	valid_0's auc: 0.746412
[9098]	valid_0's auc: 0.746411
[9099]	valid_0's auc: 0.746412
[9100]	valid_0's auc: 0.746414
[9101]	valid_0's auc: 0.74641
[9102]	valid_0's auc: 0.746411
[9103]	valid_0's auc: 0.746412
[9104]	valid_0's auc: 0.746412
[9105]	valid_0's auc: 0.746414
[9106]	valid_0's auc: 0.746414
[9107]	valid_0's auc: 0.746414
[9108]	valid_0's auc: 0.746412
[9109]	valid_0's auc: 0.746413
[9110]	valid

[9344]	valid_0's auc: 0.746455
[9345]	valid_0's auc: 0.746456
[9346]	valid_0's auc: 0.746454
[9347]	valid_0's auc: 0.746454
[9348]	valid_0's auc: 0.746453
[9349]	valid_0's auc: 0.74645
[9350]	valid_0's auc: 0.746448
[9351]	valid_0's auc: 0.746447
[9352]	valid_0's auc: 0.746447
[9353]	valid_0's auc: 0.746446
[9354]	valid_0's auc: 0.746444
[9355]	valid_0's auc: 0.746448
[9356]	valid_0's auc: 0.746449
[9357]	valid_0's auc: 0.746449
[9358]	valid_0's auc: 0.746451
[9359]	valid_0's auc: 0.746454
[9360]	valid_0's auc: 0.746453
[9361]	valid_0's auc: 0.746454
[9362]	valid_0's auc: 0.746452
[9363]	valid_0's auc: 0.746455
[9364]	valid_0's auc: 0.746456
[9365]	valid_0's auc: 0.746457
[9366]	valid_0's auc: 0.746458
[9367]	valid_0's auc: 0.746456
[9368]	valid_0's auc: 0.746455
[9369]	valid_0's auc: 0.746451
[9370]	valid_0's auc: 0.746449
[9371]	valid_0's auc: 0.74645
[9372]	valid_0's auc: 0.746452
[9373]	valid_0's auc: 0.746451
[9374]	valid_0's auc: 0.746451
[9375]	valid_0's auc: 0.746451
[9376]	val

[9610]	valid_0's auc: 0.746504
[9611]	valid_0's auc: 0.746504
[9612]	valid_0's auc: 0.746505
[9613]	valid_0's auc: 0.746506
[9614]	valid_0's auc: 0.746508
[9615]	valid_0's auc: 0.74651
[9616]	valid_0's auc: 0.746511
[9617]	valid_0's auc: 0.746508
[9618]	valid_0's auc: 0.746506
[9619]	valid_0's auc: 0.746505
[9620]	valid_0's auc: 0.746505
[9621]	valid_0's auc: 0.746507
[9622]	valid_0's auc: 0.746508
[9623]	valid_0's auc: 0.746506
[9624]	valid_0's auc: 0.746507
[9625]	valid_0's auc: 0.746508
[9626]	valid_0's auc: 0.746508
[9627]	valid_0's auc: 0.746507
[9628]	valid_0's auc: 0.746506
[9629]	valid_0's auc: 0.74651
[9630]	valid_0's auc: 0.746511
[9631]	valid_0's auc: 0.746512
[9632]	valid_0's auc: 0.746511
[9633]	valid_0's auc: 0.746512
[9634]	valid_0's auc: 0.746512
[9635]	valid_0's auc: 0.746512
[9636]	valid_0's auc: 0.746511
[9637]	valid_0's auc: 0.746516
[9638]	valid_0's auc: 0.746518
[9639]	valid_0's auc: 0.746518
[9640]	valid_0's auc: 0.746518
[9641]	valid_0's auc: 0.746519
[9642]	val

[9875]	valid_0's auc: 0.746477
[9876]	valid_0's auc: 0.746479
[9877]	valid_0's auc: 0.746482
[9878]	valid_0's auc: 0.746482
[9879]	valid_0's auc: 0.746483
[9880]	valid_0's auc: 0.746489
[9881]	valid_0's auc: 0.746488
[9882]	valid_0's auc: 0.746488
[9883]	valid_0's auc: 0.746489
[9884]	valid_0's auc: 0.74649
[9885]	valid_0's auc: 0.746489
[9886]	valid_0's auc: 0.746488
[9887]	valid_0's auc: 0.746486
[9888]	valid_0's auc: 0.746486
[9889]	valid_0's auc: 0.746487
[9890]	valid_0's auc: 0.746487
[9891]	valid_0's auc: 0.746487
[9892]	valid_0's auc: 0.74649
[9893]	valid_0's auc: 0.746487
[9894]	valid_0's auc: 0.74649
[9895]	valid_0's auc: 0.746491
[9896]	valid_0's auc: 0.746491
[9897]	valid_0's auc: 0.746489
[9898]	valid_0's auc: 0.746491
[9899]	valid_0's auc: 0.746488
[9900]	valid_0's auc: 0.746485
[9901]	valid_0's auc: 0.746486
[9902]	valid_0's auc: 0.746487
[9903]	valid_0's auc: 0.746486
[9904]	valid_0's auc: 0.746485
[9905]	valid_0's auc: 0.746484
[9906]	valid_0's auc: 0.746482
[9907]	vali

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
        learning_rate=0.05, max_depth=-1, min_child_samples=20,
        min_child_weight=50, min_split_gain=0.0, n_estimators=10000,
        n_jobs=-1, num_leaves=31, objective='binary', random_state=2018,
        reg_alpha=0.0, reg_lambda=1, silent=True, subsample=0.7,
        subsample_for_bin=200000, subsample_freq=1)

In [73]:
clf.booster_.save_model('model_0.8.txt')

In [65]:
res=pd.DataFrame()
res['aid']=predict['aid'].tolist()
res['uid']=predict['uid'].tolist()

In [78]:
%%time
result = clf.predict_proba(test_x)[:,1]

CPU times: user 3h 50min 4s, sys: 1min 5s, total: 3h 51min 10s
Wall time: 3min 27s


In [81]:
res['score'] = result
res['score'] = res['score'].apply(lambda x: float('%.6f' % x))

In [83]:
res.to_csv('./submission.csv', index=False)